# Exercise SoSe 2024
We want you to illustrate data. For questions, there are three Q and A session coming up. We recommend taking part in these Q and A sessions and be prepared.

* Results must be visualized
* Describe what you do (code)

**You are not allowed to use data already used in our Jupyter Notebooks!** We provide Links in the Jupyter Notebook for help. Take a look at the resource chapter in Moodle, there you many links, that can help. 

Remember, this is an exam admission task. We expect short, precise implementations. Sections/distances should be chosen so that your computer does not need too long computing times.  

Useful links:
* The actual Map [Open Street Map](https://www.openstreetmap.org/search?query=Aachen#map=12/50.7597/6.0964&layers=D)
* Open Street Map tag information for querying [Wind](https://wiki.openstreetmap.org/wiki/Tag:generator:source%3Dwind), [Water](https://wiki.openstreetmap.org/wiki/Tag:plant:source%3Dhydro), [Solar](https://wiki.openstreetmap.org/wiki/Tag:generator:source=solar?uselang=en), [Geothermal](https://wiki.openstreetmap.org/wiki/Tag:plant:source%3Dgeothermal), [Biomass](https://wiki.openstreetmap.org/wiki/DE:Tag:generator:source%3Dbiomass)


## How to proceed
* Select your favourite spot in Aachen and mark it on an interactive map using an icon of your choice.
* Search for one renewable energy resource position in and around the selected city and visualize the data in your interactive map using the library Python Ipyleaflet using for example an icon. 
* Finally, calculate the shortest path between your favourite spot in Aachen and the renewable energy resource and visualize it as well in your interactive map. 


In [ ]:
#Space for your code and text, add as many cells as you need!

## Submission
**FINAL Submission 23.06.2024 for your Jupyter Notebook**  
Required for exam admission.
Please submit:  
* **THIS** juypter notebook file filled with your code
* All used source/ created/ data files 

Submit as **ZIP** file, named like: **surname_student-id_course_year** in **RWTHMoodle**.
If you have worked in pairs, please name the file as follows: **surname1_student-id1_surname2_student-id2_course_year**